In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import datasets
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import random
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
%matplotlib inline


In [2]:
import sklearn
sklearn.__version__

'0.21.3'

In [3]:
df = pd.read_csv('..no show only numeric.csv')
df1 = df.copy()

In [4]:
df1.select_dtypes('object')

,ScheduledDay,AppointmentDay,Neighbourhood,SD_date
0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,JARDIM DA PENHA,2016-04-29 00:00:00+00:00
1,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,JARDIM DA PENHA,2016-04-29 00:00:00+00:00
2,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,MATA DA PRAIA,2016-04-29 00:00:00+00:00
3,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,PONTAL DE CAMBURI,2016-04-29 00:00:00+00:00
4,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,JARDIM DA PENHA,2016-04-29 00:00:00+00:00
...,...,...,...,...
110522,2016-05-03 09:15:35+00:00,2016-06-07 00:00:00+00:00,MARIA ORTIZ,2016-05-03 00:00:00+00:00
110523,2016-05-03 07:27:33+00:00,2016-06-07 00:00:00+00:00,MARIA ORTIZ,2016-05-03 00:00:00+00:00
110524,2016-04-27 16:03:52+00:00,2016-06-07 00:00:00+00:00,MARIA ORTIZ,2016-04-27 00:00:00+00:00
110525,2016-04-27 15:09:23+00:00,2016-06-07 00:00:00+00:00,MARIA ORTIZ,2016-04-27 00:00:00+00:00


Summary after Data Visualization
- Age less than 0 and more than 100 should be drop
- days between where appoitment was earlier than schedule should be drop
- Unnamed:0 can be drop
- Appoitment ID is unique and can be use in index

Data for split 
- we should avoid that we have the same person in train and test
- comparision for diffrent train and test


## Split train/test



### whole set

In [5]:
X_train, X_test  = train_test_split(df,test_size=0.33,random_state=42,stratify = df['No-show'])


print(f'Train set with attributes has {X_train.shape[0]} rows and Test set has {X_test.shape[0]} rows')

Train set with attributes has 74053 rows and Test set has 36474 rows


### target set balance

In [6]:
percent_YES_train = X_train['No-show'].value_counts()[1]/X_train['No-show'].value_counts().sum()
percent_YES_test = X_test['No-show'].value_counts()[1]/X_test['No-show'].value_counts().sum()
#print('We have ca. 20% YES and 80% NO in both sets')

### gander balanced sets

In [7]:
X_train.Gender.value_counts()
percent_Woman_train = round(X_train.Gender.value_counts()/X_train.Gender.value_counts().sum()*100)
percent_Woman_test = round(X_test.Gender.value_counts()/X_test.Gender.value_counts().sum()*100)

print('Our train and test sets with attributes have 65% woman in both of them')
percent_Woman_train 

Our train and test sets with attributes have 65% woman in both of them


0    65.0
1    35.0
Name: Gender, dtype: float64

## stratified suffle split 

In [8]:
df_without_target = df.drop('No-show',axis=1)
target = df['No-show']
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)

for train_index,test_index in split.split(df_without_target,target):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_s, X_test_s = df1.iloc[train_index], df1.iloc[test_index]
    y_train_s, y_test_s = target[train_index], target[test_index]

## unique patients in train and test

In [9]:
patient_list = df.PatientId.unique()
print(patient_list)
random.seed(42)
np.random.shuffle(patient_list)

print(patient_list)

size_of_test = round(len(patient_list)*0.25)
size_of_train = round(len(patient_list)*0.75)

patient_train = patient_list[15575:]
patient_test =  patient_list[:15575]

train = df[df['PatientId'].isin(patient_train)]
test = df[df['PatientId'].isin(patient_test)]


[2.98724998e+13 5.58997777e+14 4.26296230e+12 ... 7.26331493e+13
 9.96997666e+14 1.55766317e+13]
[4.84616270e+14 4.17133365e+12 1.49593154e+13 ... 5.17878585e+13
 2.39695957e+13 3.78677738e+13]


### train-test split comparision

In [10]:
sampling_bias_comparision = pd.DataFrame()
whole_set_gender = df.Gender.value_counts()/len(df1)
sampling_bias_comparision['overall_gender'] = whole_set_gender

sampling_bias_comparision_target = pd.DataFrame()
sampling_bias_comparision_target['target_oberall'] = target.value_counts()/len(target)

sampling_bias_comparision['train_test_split_gender_TRAIN'] = X_train.Gender.value_counts()/len(X_train)
sampling_bias_comparision['train_test_split_gender_TEST'] = X_test.Gender.value_counts()/len(X_test)

sampling_bias_comparision_target['train_test_split_target_TRAIN'] = X_train['No-show'].value_counts()/len(X_train['No-show'])
sampling_bias_comparision_target['train_test_split_target_TEST'] = X_test['No-show'].value_counts()/len(X_test['No-show'])



sampling_bias_comparision['StratifiedShufflesplit_TRAIN'] = X_train_s.Gender.value_counts()/len(X_train_s)
sampling_bias_comparision['StratifiedShufflesplit_TEST'] = X_test_s.Gender.value_counts()/len(X_test_s)

sampling_bias_comparision_target['StratifiedShufflesplit_target_TRAIN'] = y_train_s.value_counts()/len(y_train_s)
sampling_bias_comparision_target['StratifiedShufflesplit-taget_TEST'] = y_test_s.value_counts()/len(y_test_s)

sampling_bias_comparision_target['UniquePatients_TRAIN'] = train['No-show'].value_counts()/len(train)
sampling_bias_comparision_target['UniquePatients_TEST'] = test['No-show'].value_counts()/len(test)

sampling_bias_comparision_target

,target_oberall,train_test_split_target_TRAIN,train_test_split_target_TEST,StratifiedShufflesplit_target_TRAIN,StratifiedShufflesplit-taget_TEST,UniquePatients_TRAIN,UniquePatients_TEST
0,0.798067,0.798064,0.798075,0.798068,0.798064,0.797784,0.798915
1,0.201933,0.201936,0.201925,0.201932,0.201936,0.202216,0.201085


In [11]:
# we will take train and test to do analysis. We use UniquePatientSplit to create this two datasets
y_train=train['No-show']
y_test =test['No-show']
train = train.drop(['Unnamed: 0','No-show','Neighbourhood'],axis=1)
test = test.drop(['Unnamed: 0','No-show','Neighbourhood'],axis=1)


In [12]:
train1 = train.copy()

# Pipeline

In [14]:
#full pipeline
full_pipeline = FeatureUnion(transformer_list=[
    ('cat_pipeline',cat_pipeline),
    ('num_pipeline',pipeline_num)
])


train_prep = full_pipeline.fit_transform(train)
y_train = pd.factorize(y_train)[0]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
lin_reg = LinearRegression()
scores = cross_val_score(lin_reg,train_prep,y_train[0],scoring='accuracy')